In [1]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import os
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn import svm
from sklearn.model_selection import cross_val_score


files = ['result_kujou_sud/' + path for path in os.listdir('result_kujou_sud')]
count_vect = CountVectorizer(input='filename')
X_train_counts = count_vect.fit_transform(files)


tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

features = count_vect.get_feature_names()
print(features[0:100])

X_train_tfidf = X_train_tfidf.toarray()
print(X_train_tfidf.max())
all_X = X_train_tfidf
all_y = np.loadtxt('only_suuti_label.csv', delimiter = '\n', dtype = float)
#print(all_y)
#print(type(all_X))
#print(type(all_y))
print(all_X.shape)
#print(count_vect.vocabulary_)

['11０', '1０', '1００', '1０００', '21', '2０', '2００', '2０１７', '82０', 'お客様の声', 'お金', 'しろ', 'ずれ', 'みれ', 'アウディ', 'アクセラ', 'アップ', 'アドバイス', 'アルミホイール', 'アンケート', 'アール', 'イング', 'ウェブ', 'エスカレーション', 'カスタマー', 'キュー', 'クレジット', 'グループ', 'コスト', 'コメント', 'コメント欄', 'コース', 'コールセンター', 'サポート', 'サービス', 'システム', 'ショップ', 'ストップ', 'センター', 'ダイヤル', 'ダウン', 'テニス', 'テル', 'ディーラー', 'デメリット', 'データ', 'ドア', 'ナンバー', 'ノーカウント', 'バイク', 'バンパー', 'ファックス', 'フォロー', 'フリーコール', 'フリーダイヤル', 'プラン', 'プリウス', 'プレー', 'プロテクト', 'ペルテ', 'ページ', 'ホーム', 'ポリープ', 'ポロ', 'マーク', 'ミス', 'メリット', 'メール', 'レジャー', 'レター', 'ワン', '一律', '一括', '一方的', '一時停止', '一環', '一般', '一言', '上から目線', '上記', '上長', '下記', '不備', '不十分', '不可', '不在', '不安', '不快', '不満', '不親切', '不足', '中学生', '中断', '中止', '了承', '了解', '予定', '争点', '事務的', '事実関係']
0.7563927854802805
(79, 615)


In [2]:
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from tqdm import tqdm

loo = LeaveOneOut()
entire_count = loo.get_n_splits(all_X) 
p = np.array([1.0])
n = np.array([0.0])
print(p[0])
best = []


for c in [ 1000.0]:
    pipe  = make_pipeline(StandardScaler(), SVC(C =c, gamma=0.001,kernel = "rbf",random_state = 2))
    correct_answer_count = 0 
    fn=0
    fp=0
    tp=0
    tn=0            
    for train_index, test_index in loo.split(all_X):
        #print(train_index, test_index)
        x_train, x_test = all_X[train_index], all_X[test_index]
        y_train, y_test = all_y[train_index], all_y[test_index]
        pipe.fit(x_train, y_train)  # 学習させる
        result = pipe.predict(x_test)  # テストデータからラベルを予測する
        if result == y_test:  # ラベルと元々のラベルが一致していれば+1
            correct_answer_count += 1
        if result != y_test and result[0] == n[0]:
            fn += 1
        elif result != y_test and result[0] == p[0]:
            fp +=1
        elif result == y_test and result[0] == p[0]:
            tp += 1
        elif result == y_test and result[0] == n[0]:
            tn += 1

    accuracy = (float(tp+tn)/float(tp+tn+fp+fn))
    true_positive_rate = (float(tp)/float(tp+fn))
    true_negative_rate = (float(tn)/float(tn + fp))
    false_positive_rate = (float(fp)/float(fp + tn))
    false_negative_rate = (float(fn)/float(fn + tp))
    if accuracy >0.6:
        best.append([c])

    print("SVM_C = %s    " % (c))
    print("Accuracy:%s" % accuracy)
    print("TP_rate: %s" % true_positive_rate)
    print("TN_rate: %s" % true_negative_rate)
    print("FP_rate: %s" % false_positive_rate)
    print("FN_rate: %s" % false_negative_rate)
    rate = (float(correct_answer_count) / float(entire_count))  # 正解率を計算
    print("Accuracy : %s" % str(rate))  # 正解率を出力
    print("F1: %s" % (2*(true_positive_rate*(float(tp)/float(tp+fp)))/(true_positive_rate+(float(tp)/float(tp+fp)))))
    print("")


print(tp)
print("kpca_best_param: %s" %best)
# scores = cross_val_score(pipe, all_X, all_y , cv= 25)
# print("Cross-Validation scores: {}".format(scores))
# print("Average score: {}".format(np.mean(scores)))


# pred_y2 = pipe.predict(test_X)
# print(test_y)
# print(pred_y2)
# print(f1_score(test_y, pred_y2, average='micro'))



#from sklearn.metrics import confusion_matrix
#print(confusion_matrix(test_y, pred_y2))
#if result != y_test and float(result) ==　0:
   #     fn += 1
#elif result != y_test and float(result) == 1:
   #     fp +=1
#elif result == y_test and float(result) ==1:
 #       tp += 1
#elif result == y_test and float(result) == 0:
   #     fn += 1

1.0
SVM_C = 1000.0    
Accuracy:0.6075949367088608
TP_rate: 0.6829268292682927
TN_rate: 0.5263157894736842
FP_rate: 0.47368421052631576
FN_rate: 0.3170731707317073
Accuracy : 0.6075949367088608
F1: 0.6436781609195402

28
kpca_best_param: [[1000.0]]
